In [1]:
import pandas as pd
import numpy as np
import json
import requests
from time import sleep
from pandas.io.json import json_normalize

In [223]:
d2 = pd.read_json('d2m.json')
d3 = pd.read_json('d2m1.json')
final_df = pd.read_json('final_table.json')

In [227]:
d2m = d2['match_id']
match_ids = []
for id in d2m:
    match_ids.append(id)
len(match_ids)

100

In [228]:
d3m = d3['match_id']
for id in d3m:
    match_ids.append(id)

In [229]:
match_ids = list(set(match_ids))


In [230]:
main_features = ['players', 'radiant_gold_adv', 'radiant_xp_adv','radiant_win']

In [236]:
d2dfc = pd.DataFrame()
df_arrays = [f'd2df{i}' for i in range(1, len(match_ids))]

In [239]:
for i in range(0, len(match_ids) - 1):
    if i // 59 == 0:
        sleep(60)
        
    URL = f'https://api.opendota.com/api/matches/{match_ids[i]}'
    
    df_arrays[i] = pd.json_normalize(requests.get(URL).json())
    df_arrays[i]['radiant_win'] = df_arrays[i]['radiant_win'].apply(lambda x: int(x))
    df_arrays[i] = df_arrays[i][main_features]
    
    df_arrays[i]['radiant_gold_adv_7'] = df_arrays[i]['radiant_gold_adv'].iloc[0][6]
    df_arrays[i]['radiant_gold_adv_9'] = df_arrays[i]['radiant_gold_adv'].iloc[0][8]
    df_arrays[i]['radiant_gold_adv_10'] = df_arrays[i]['radiant_gold_adv'].iloc[0][9]
    
    df_arrays[i]['radiant_xp_adv_7'] = df_arrays[i]['radiant_xp_adv'].iloc[0][6]
    df_arrays[i]['radiant_xp_adv_9'] = df_arrays[i]['radiant_xp_adv'].iloc[0][8]
    df_arrays[i]['radiant_xp_adv_10'] = df_arrays[i]['radiant_xp_adv'].iloc[0][9]
    
    array_actions = []
    
    for dictionary in df_arrays[i]['players'][0]:
        array_actions.append(dictionary['actions_per_min'])
    try:
        df_arrays[i]['apmr_apmd'] = sum(array_actions[:5]) / sum(array_actions[5:])
    except ZeroDivisionError:
        df_arrays[i]['apmr_apmd'] = 0
    else:
        d2dfc = pd.concat([d2dfc, df_arrays[i]])
    
    
d2dfc = d2dfc.set_index(np.arange(0, len(d2dfc)))    
d2dfc




,players,radiant_gold_adv,radiant_xp_adv,radiant_win,radiant_gold_adv_7,radiant_gold_adv_9,radiant_gold_adv_10,radiant_xp_adv_7,radiant_xp_adv_9,radiant_xp_adv_10,apmr_apmd
0,"[{'match_id': 6424583170, 'player_slot': 0, 'a...","[0, -515, -960, -1175, -1236, -1724, -1678, -2...","[0, 7, -73, -252, -689, -1227, -1329, -1513, -...",0,-1678,-2759,-3032,-1329,-2399,-2758,1.622294
1,"[{'match_id': 6425768476, 'player_slot': 0, 'a...","[0, -464, -247, -21, -218, 187, 502, 1947, 190...","[0, 5, 233, 345, -201, -35, -43, 1477, 1978, 3...",1,502,1900,2584,-43,1978,3196,1.148637
2,"[{'match_id': 6424722973, 'player_slot': 0, 'a...","[0, 39, -84, -17, -195, -573, 523, 701, 1466, ...","[0, -34, -143, -295, -348, -811, -224, -640, 4...",1,523,1466,1967,-224,492,1063,1.246743
3,"[{'match_id': 6424488988, 'player_slot': 0, 'a...","[0, 559, 325, 255, 186, -161, -691, -693, -572...","[0, 312, -183, 97, -8, -435, -1369, -998, -825...",1,-691,-572,-465,-1369,-825,-708,1.133712
4,"[{'match_id': 6428580900, 'player_slot': 0, 'a...","[0, 523, 520, 590, 184, -397, 195, 942, -257, ...","[0, 120, 108, -90, -387, -935, -127, 597, -407...",0,195,-257,217,-127,-407,188,1.148254
...,...,...,...,...,...,...,...,...,...,...,...
160,"[{'match_id': 6424204273, 'player_slot': 0, 'a...","[0, 218, -152, -25, 269, 587, 1052, 698, 1306,...","[0, 130, -290, -208, 70, 598, 1569, 2180, 3392...",1,1052,1306,1195,1569,3392,4203,0.862464
161,"[{'match_id': 6429237232, 'player_slot': 0, 'a...","[0, 255, 794, 1238, 1180, 1369, 1556, 1867, 27...","[0, 102, 396, 515, 1280, 1111, 1222, 1097, 190...",0,1556,2720,2016,1222,1901,1428,0.937931
162,"[{'match_id': 6425819126, 'player_slot': 0, 'a...","[0, 43, -693, -486, -624, -770, -1007, -1676, ...","[0, -45, -789, -595, -568, -307, -541, -773, -...",0,-1007,-1412,-1755,-541,-531,-1370,0.879853
163,"[{'match_id': 6429507064, 'player_slot': 0, 'a...","[491, 926, 1131, 1644, 1515, 1281, 1212, 1262,...","[200, 5, 872, 1225, 1530, 527, 787, 499, -32, ...",1,1212,898,1166,787,-32,86,0.999155


In [240]:
d2dfc.drop(['players', 'radiant_gold_adv', 'radiant_xp_adv'], axis = 1, inplace = True)

    

In [282]:
d2dfc.to_json('\final_table_under10min.json')

In [241]:
d2dfc

,radiant_win,radiant_gold_adv_7,radiant_gold_adv_9,radiant_gold_adv_10,radiant_xp_adv_7,radiant_xp_adv_9,radiant_xp_adv_10,apmr_apmd
0,0,-1678,-2759,-3032,-1329,-2399,-2758,1.622294
1,1,502,1900,2584,-43,1978,3196,1.148637
2,1,523,1466,1967,-224,492,1063,1.246743
3,1,-691,-572,-465,-1369,-825,-708,1.133712
4,0,195,-257,217,-127,-407,188,1.148254
...,...,...,...,...,...,...,...,...
160,1,1052,1306,1195,1569,3392,4203,0.862464
161,0,1556,2720,2016,1222,1901,1428,0.937931
162,0,-1007,-1412,-1755,-541,-531,-1370,0.879853
163,1,1212,898,1166,787,-32,86,0.999155


,radiant_win,radiant_gold_adv_10,radiant_gold_adv_13,radiant_gold_adv_15,radiant_xp_adv_10,radiant_xp_adv_13,radiant_xp_adv_15,apmr_apmd
0,0,-3032,-972,-1244,-2758,-553,-93,1.622294
1,1,-465,-944,-2222,-708,-1346,-4437,1.133712
2,1,2584,4700,6865,3196,5874,8788,1.148637
3,1,1967,3731,4755,1063,3117,4548,1.246743
4,0,217,-1081,-1413,188,417,-865,1.148254
...,...,...,...,...,...,...,...,...
161,0,2016,3359,4855,1428,3376,5732,0.937931
162,0,-1755,-1911,-771,-1370,-2096,-476,0.879853
163,1,1166,1405,3095,86,1008,4590,0.999155
164,1,-4711,-4970,-6049,-3575,-3648,-4577,1.157113


In [242]:
y = d2dfc['radiant_win']
X = d2dfc.drop('radiant_win', axis=1)

In [243]:
from sklearn.model_selection import train_test_split

In [264]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [279]:
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = preprocessing.StandardScaler().fit(X_train)
scaler.mean_
scaler.scale_
X_scaled = scaler.transform(X_train)
pipe = make_pipeline(StandardScaler(), SVC())
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)


0.6904761904761905